In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn import model_selection
from sklearn import metrics
from sklearn import svm
from time import time
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import math
import random
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("train.csv")

In [3]:
pd.set_option("display.max_columns",30)
data.head(3)

,ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,4,April,23,Scientist,19114.12,NaN,3,4,3,4,5,4.0,6.27,4.0,Good,809.98,31.377862,268,No,49.574949,199.458074,223.451310,Good
1,5,May,23,Scientist,19114.12,1824.843333,3,4,3,4,6,NaN,11.27,4.0,Good,809.98,24.797347,269,No,49.574949,41.420153,341.489231,Good
2,6,June,23,Scientist,19114.12,NaN,3,4,3,4,8,4.0,9.27,4.0,Good,809.98,27.262259,270,No,49.574949,62.430172,340.479212,Good


In [4]:
data = data.drop(data[data.Occupation=='_______'].index)
data = data.drop(data[data.Credit_Mix=='_'].index)

In [5]:
data = data.dropna(axis=0,how='any')
data.reset_index()

,index,ID,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance,Credit_Score
0,3,10,February,28,Teacher,34847.84,3037.986667,2,4,6,1,7,1.0,7.42,2.0,Good,605.03,38.550848,320,No,18.816215,40.391238,484.591214,Good
1,4,13,May,28,Teacher,34847.84,3037.986667,2,4,6,1,3,1.0,6.42,2.0,Good,605.03,34.977895,323,No,18.816215,130.115420,444.867032,Good
2,5,14,June,28,Teacher,34847.84,3037.986667,2,4,6,1,3,0.0,5.42,2.0,Good,605.03,33.381010,324,No,18.816215,43.477190,481.505262,Good
3,7,16,August,28,Teacher,34847.84,3037.986667,2,4,6,1,3,4.0,5.42,2.0,Good,605.03,32.933856,326,No,18.816215,218.904344,356.078109,Good
4,9,18,February,34,Engineer,143162.64,12187.220000,1,5,8,3,13,6.0,7.10,3.0,Good,1303.01,41.702573,214,No,246.992320,232.860384,998.869297,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17454,25935,99980,April,49,Writer,37188.10,3097.008333,1,4,5,3,3,12.0,5.38,3.0,Good,620.64,36.979007,360,NM,84.205949,215.486386,300.008498,Good
17455,25936,99983,July,50,Writer,37188.10,3097.008333,1,4,5,3,7,13.0,5.38,3.0,Good,620.64,38.850680,363,No,84.205949,82.825572,412.669312,Good
17456,25937,99986,February,28,Architect,20002.88,1929.906667,10,8,29,5,33,25.0,23.31,9.0,Bad,3571.70,39.772607,69,Yes,12112.000000,148.275233,273.750661,Poor
17457,25938,99987,March,28,Architect,20002.88,1929.906667,10,8,29,5,33,25.0,18.31,9.0,Bad,3571.70,24.713861,70,Yes,60.964772,76.282793,345.743101,Poor


In [6]:
from sklearn.preprocessing import LabelEncoder
data['Occupation'] = LabelEncoder().fit_transform(data['Occupation'])
data['Credit_Mix'] = LabelEncoder().fit_transform(data['Credit_Mix'])
data['Payment_of_Min_Amount'] = LabelEncoder().fit_transform(data['Payment_of_Min_Amount'])
data['Month'] = LabelEncoder().fit_transform(data['Month'])
data['Credit_Score'] = LabelEncoder().fit_transform(data['Credit_Score'])

In [7]:
data = data.drop(data[data.Age>100].index)

In [8]:
data = data.sample(n=10000, random_state = 42)

In [9]:
predictor = data.columns[1:22]

In [10]:
from sklearn import preprocessing
predictors = data.columns[1:22]
X = data[predictor]
y = data.Credit_Score
#使用SMOTE采样，从训练集随机同类的两近邻样本合成一个新的样本，合成新样本的过采样方法可以降低过拟合的风险。
sm = SMOTE(random_state=42)
X, y = sm.fit_resample(X,y)

In [11]:
X_train,X_test,y_train,y_test = model_selection.train_test_split(X,y,test_size=0.25,random_state=42)

In [12]:
X_train.head(3)

,Month,Age,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Monthly_Balance
9233,1,38,13,8475.64,794.303333,10,10,22,3,50,25.0,5.76,11.0,0,2033.14,36.940310,123,2,12.792123,98.348957,258.289254
1616,1,26,4,18115.21,1327.600833,9,8,18,6,15,10.0,9.18,11.0,2,1440.76,36.856242,233,2,62.846716,46.329711,313.583656
2846,0,51,10,31534.94,2426.911667,5,6,1,0,4,0.0,5.99,4.0,1,1258.83,26.974031,354,1,0.000000,25.088514,457.602652


In [13]:
X_train.to_csv("X_train.csv")
X_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv("y_test.csv")